In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Normalize the pixel values between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert the labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 100)
y_test = keras.utils.to_categorical(y_test, 100)


In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"E:\imbcifar\train"

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.11it/s]

X_train shape: (29009, 32, 32, 3)
Y_train shape: (29009,)


In [1]:
from PIL import Image
import numpy as np
import os

def read_and_process_images(base_dir):
    images = []
    # Iterate through all subdirectories
    for subdir, dirs, files in os.walk(base_dir):
        for file in files:
            # Construct the full file path
            filepath = os.path.join(subdir, file)
            try:
                # Open the image
                with Image.open(filepath) as img:
                    # Convert to RGB
                    img = img.convert('RGB')
                    # Resize to 32x32
                    img = img.resize((32, 32), Image.ANTIALIAS)
                    # Convert to numpy array and scale
                    img_array = np.asarray(img, dtype=np.float32) / 255
                    images.append(img_array)
            except IOError:
                # Handle the case where the file could not be opened as an image
                print(f"Cannot open {file} as an image.")
                
    return images

base_dir = r"D:\datasets\Underwater_Image\WHOI\archive\dataset_pm\training"
X_train = read_and_process_images(base_dir)

C:\Users\shaif\AppData\Local\Temp\ipykernel_16216\1555111213.py:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((32, 32), Image.ANTIALIAS)


Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.
Cannot open desktop.ini as an image.


In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Define the paths to your dataset folders
train_dataset_dir = r"D:\datasets\Underwater_Image\LIMUC (Labeled Images for Ulcerative Colitis)\train_and_validation_sets\train_and_validation_sets"
test_dataset_dir = r"D:\datasets\Underwater_Image\LIMUC (Labeled Images for Ulcerative Colitis)\test_set\test_set"

# Initialize empty lists for X_train, Y_train, X_test, and Y_test
X_train = []
Y_train = []
X_test = []
Y_test = []

# Initialize an empty list to store categorical labels
categorical_labels = []

# Define a function to read and preprocess images
def process_images(folder_path, label, is_train_set=True):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and filename.endswith(".bmp"):  # Check if it's a file and ends with .bmp
            # Open and resize the image to (32, 32, 3)
            img = Image.open(file_path)
            img = img.resize((32, 32))
            img = img.convert("RGB")
            
            # Convert image data to a NumPy array
            img_array = np.array(img).astype('float32')  # Convert to float
            
            # Normalize the image data (optional)
            img_array /= 255.0  # Normalize pixel values to [0, 1]
            
            # Append the image data to the appropriate list
            if is_train_set:
                X_train.append(img_array)
                Y_train.append(label)  # Append the numerical label
            else:
                X_test.append(img_array)
                Y_test.append(label)  # Append the numerical label
            
            # Append the label for categorical encoding
            categorical_labels.append(label)  # Append the numerical label

# List the folders inside the training dataset directory
train_folders = os.listdir(train_dataset_dir)

# Create a label encoder for categorical labels
label_encoder = LabelEncoder()

# Loop through the training folders and process images
for label, folder_name in enumerate(train_folders):
    folder_path = os.path.join(train_dataset_dir, folder_name)
    if os.path.isdir(folder_path):  # Check if it's a directory
        process_images(folder_path, label)

# List the folders inside the test dataset directory
test_folders = os.listdir(test_dataset_dir)

# Loop through the test folders and process images
for label, folder_name in enumerate(test_folders):
    folder_path = os.path.join(test_dataset_dir, folder_name)
    if os.path.isdir(folder_path):  # Check if it's a directory
        process_images(folder_path, label, is_train_set=False)
        
        
        
x_train = np.concatenate((X_train, X_test), axis=0)
# Check the shape of X_train, Y_train_categorical, X_test, and Y_test_categorical
print("Shape of X_train:", np.shape(x_train))

Shape of X_train: (11276, 32, 32, 3)


In [ ]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"G:\datasets\Underwater_Image\WHOI\archive\dataset_pm\training"

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

 58%|███████████████████████████████████████████████▍                                  | 11/19 [03:50<04:22, 32.87s/it]

In [1]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

# Load the iNaturalist dataset
dataset_name = 'i_naturalist2018'
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name,
    split=['train', 'test'],
    as_supervised=True,  # Get the (image, label) pairs
    with_info=True
)

# Function to preprocess images and labels
def preprocess(image, label):
    image = tf.image.resize(image, (32, 32))
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    label = tf.one_hot(label, depth=ds_info.features['label'].num_classes)  # One-hot encode labels
    return image, label

# Preprocess train and test datasets
ds_train = ds_train.map(preprocess).batch(32).shuffle(buffer_size=1000)
ds_test = ds_test.map(preprocess).batch(32)

# Convert the TensorFlow dataset to numpy arrays for x_train and y_train
def dataset_to_numpy(ds):
    images = []
    labels = []
    for image_batch, label_batch in ds:
        images.append(image_batch.numpy())
        labels.append(label_batch.numpy())
    return np.concatenate(images), np.concatenate(labels)

# Convert train and test datasets
x_train, y_train = dataset_to_numpy(ds_train)
x_test, y_test = dataset_to_numpy(ds_test)

# Verify shapes
print(f'x_train shape: {x_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'x_test shape: {x_test.shape}')
print(f'y_test shape: {y_test.shape}')


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

UnknownError: Failed to WriteFile: C:\Users\shaif\tensorflow_datasets\downloads\ml-inat-competi-dataset.s3_2018_test201y0Id8aFmoOuWp0nWu62p_H0DreGZ8ygzUUVFwolgzZ8.tar.gz.tmp.c6ce1278ee294654a198b9a8699be117\test2018.tar.gz : There is not enough space on the disk.
; operation in progress

In [4]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
import glob

target_size = (32, 32)  # Change the values as per your requirement
# Load the pre-trained ResNet50 model with modified input shape
model = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(target_size[0], target_size[1], 3))

In [5]:
import pandas as pd
# Define the file path
file_path = r"D:\feature.csv"
# Read the tab-separated CSV file into a DataFrame
df = pd.read_csv(file_path, delimiter='\t')
columns_to_drop = [df.columns[0], df.columns[-1]]
data = df.drop(columns_to_drop, axis=1)
# Display the head of the DataFrame
data.head(1)

,Feature 0,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,...,Feature 2038,Feature 2039,Feature 2040,Feature 2041,Feature 2042,Feature 2043,Feature 2044,Feature 2045,Feature 2046,Feature 2047
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.009121,0.0,0.0,0.0,0.0,3.285227,0.0,0.0


In [4]:
np.shape(X_train)

(4357, 32, 32, 3)

In [7]:
from PIL import Image
import numpy as np
ft = model.predict(np.array(X_train).astype("float32"))

In [8]:
from sklearn.cluster import MiniBatchKMeans

n_clusters = 400
batch_size = 100
max_iter = 100

kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size, max_iter=max_iter)
kmeans.fit(ft)
# Retrieve the cluster centers
ct = kmeans.cluster_centers_.tolist()

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [9]:
from tqdm import tqdm

combo_list = []  # Initialize combo_list
tot_dist = []
# Iterate over ft
for i in tqdm(range(len(ct))):
    distances = []

    # Calculate distances for each row in data
    for index, row in data.iterrows():
        row_array = row.to_numpy()  # Convert row to numpy array
        distance = np.linalg.norm(ct[i] - row_array)  # Calculate Euclidean distance
        distances.append(distance)

    tot_dist.append(distances)
print(np.shape(tot_dist))
#new_shape = (1000, 30)
#re_dist = np.transpose(tot_dist, (1, 0))
re_dist = tot_dist
print(np.shape(re_dist))

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [01:00<00:00,  6.60it/s]

(400, 1000)
(400, 1000)


In [14]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\dist_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(re_dist, file)

print("Combo list saved successfully.")

Combo list saved successfully.


In [13]:
len(index_list)

NameError: name 'index_list' is not defined

In [11]:
import pickle

# Loading the list from the pickle file
file_path = r"D:\index_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    index_list = pickle.load(file)

print("Combo list loaded successfully.")
print(index_list)


Combo list loaded successfully.
[1, 2, 3, 4, 5, 6, 11, 14, 18, 20, 21, 22, 23, 24, 27, 29, 31, 33, 34, 41, 42, 45, 47, 49, 50, 58, 65, 66, 70, 72, 73, 74, 75, 76, 78, 81, 83, 85, 86, 93, 96, 98, 102, 103, 105, 107, 108, 109, 115, 116, 117, 123, 127, 128, 129, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143, 147, 149, 152, 153, 156, 157, 165, 166, 167, 179, 181, 183, 188, 190, 195, 197, 204, 205, 206, 214, 215, 216, 217, 221, 223, 224, 229, 232, 234, 238, 239, 240, 241, 244, 247, 252, 256, 257, 259, 272, 274, 276, 277, 278, 280, 288, 291, 293, 297, 299, 302, 305, 306, 317, 318, 320, 321, 322, 324, 325, 326, 330, 331, 332, 342, 343, 344, 346, 351, 352, 353, 362, 363, 368, 369, 373, 375, 378, 379, 384, 386, 388, 393, 397, 399, 400, 403, 404, 405, 406, 408, 409, 411, 413, 414, 417, 424, 426, 432, 435, 436, 438, 442, 447, 449, 451, 453, 457, 458, 460, 465, 466, 468, 470, 472, 474, 475, 479, 484, 493, 495, 498, 500, 509, 510, 511, 512, 518, 520, 525, 527, 528, 531, 533, 535, 536, 538, 

In [5]:
df.columns

Index(['Directory', 'Feature 0', 'Feature 1', 'Feature 2', 'Feature 3',
       'Feature 4', 'Feature 5', 'Feature 6', 'Feature 7', 'Feature 8',
       ...
       'Feature 2039', 'Feature 2040', 'Feature 2041', 'Feature 2042',
       'Feature 2043', 'Feature 2044', 'Feature 2045', 'Feature 2046',
       'Feature 2047', 'Unnamed: 2049'],
      dtype='object', length=2050)

In [12]:
selected_data = df.loc[index_list, ['Name']].to_numpy().flatten()

In [14]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\combo_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(selected_data, file)

print("Combo list saved successfully.")


Combo list saved successfully.


In [15]:
import pickle

# Loading the list from the pickle file
file_path = r"D:\combo_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    loaded_combo_list = pickle.load(file)

print("Combo list loaded successfully.")
print(loaded_combo_list)


Combo list loaded successfully.
['D:/data/imagenet\\n01443537\\' 'D:/data/imagenet\\n01484850\\'
 'D:/data/imagenet\\n01491361\\' 'D:/data/imagenet\\n01494475\\'
 'D:/data/imagenet\\n01496331\\' 'D:/data/imagenet\\n01498041\\'
 'D:/data/imagenet\\n01531178\\' 'D:/data/imagenet\\n01537544\\'
 'D:/data/imagenet\\n01582220\\' 'D:/data/imagenet\\n01601694\\'
 'D:/data/imagenet\\n01608432\\' 'D:/data/imagenet\\n01614925\\'
 'D:/data/imagenet\\n01616318\\' 'D:/data/imagenet\\n01622779\\'
 'D:/data/imagenet\\n01631663\\' 'D:/data/imagenet\\n01632777\\'
 'D:/data/imagenet\\n01644373\\' 'D:/data/imagenet\\n01664065\\'
 'D:/data/imagenet\\n01665541\\' 'D:/data/imagenet\\n01685808\\'
 'D:/data/imagenet\\n01687978\\' 'D:/data/imagenet\\n01692333\\'
 'D:/data/imagenet\\n01694178\\' 'D:/data/imagenet\\n01697457\\'
 'D:/data/imagenet\\n01698640\\' 'D:/data/imagenet\\n01737021\\'
 'D:/data/imagenet\\n01751748\\' 'D:/data/imagenet\\n01753488\\'
 'D:/data/imagenet\\n01770081\\' 'D:/data/imagenet\\n01773

In [5]:
import os

file_path = r"F:\D-Video\Python"

# Check if the file exists
if os.path.exists(file_path):
    print(f"File exists: {file_path}")
else:
    print(f"File does not exist: {file_path}")

# Check if the file is readable
if os.access(file_path, os.R_OK):
    print(f"File is readable: {file_path}")
else:
    print(f"File is not readable: {file_path}")


File exists: F:\D-Video\Python
File is readable: F:\D-Video\Python
